# Predicting the Average Popularity of a Song

This notebook is used to create a predictive model for the average popularity of a song using most of the metrics Spotify provides for each song, along with the artist of the song, the album release date.
The model is created using XGBoost. The data is from the warehouse we created and was exported to an Excel file called songs_warehouse.xlsx.

## Import necessary libraries
Import the necessary libraries including pandas, train_test_split, xgboost, and mean_squared_error from sklearn.metrics.

In [19]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

## Import the data
The data is in an Excel file called songs_warehouse.xlsx. There are 3 sheets in the file: song_dimension, song_snapshot_fact, and artist_of_song. 
* The song_dimension sheet contains the metrics for each song.
* The song_snapshot_fact sheet contains the metrics for each song at a specific time.
* The artist_of_song sheet contains the artist of each song.

In [20]:
df = pd.read_excel('data/songs_warehouse.xlsx', sheet_name='song_dimension')
df.head()

,id_song,label_spotify_id,label_name,label_is_explicit,label_duration_ms,label_album_name,label_album_release_date,label_danceability,label_energy,label_key,label_loudness,label_mode,label_speechiness,label_acousticness,label_instrumentalness,label_liveness,label_valence,label_tempo,label_time_signature
0,42,03Dpt8Z4Zww4NGJb8503zb,Do They Know It's Christmas? - 2014,False,228916,Pop Christmas Songs,2018-11-09,0.626,0.541,0,-7.615,1,0.0308,0.3520,0.0,0.119,0.255,112.000999,4
1,128,0a0zPUrwviAua4IhhaYUsP,Ajándék,False,268240,Duett Karácsony,2009-01-01,0.668,0.864,0,-4.404,1,0.0343,0.0473,0.0,0.105,0.593,102.014000,4
2,182,0cVyQfDyRnMJ0V3rjjdlU3,Lil Boo Thang,False,114233,Lil Boo Thang,2023-08-18,0.850,0.699,0,-3.292,1,0.0776,0.1520,0.0,0.320,0.915,114.481003,4
3,270,0gq4UgDPGFdqpsWshU7dmv,Vanavond (Uit M'n Bol),False,166560,Vanavond (Uit M'n Bol),2022-03-25,0.799,0.705,0,-7.582,1,0.0698,0.0554,0.0,0.426,0.880,106.978996,4
4,289,0hI4TphLTs4ar0mQ8t0dLf,Мой счастливый билет,False,164023,Мой счастливый билет,2023-02-24,0.783,0.524,0,-6.772,1,0.1300,0.6330,0.0,0.189,0.470,110.035004,4


In [21]:
df2 = pd.read_excel('data/songs_warehouse.xlsx', sheet_name='song_snapshot_fact')

Note that in the artist_of_song sheet, the column artist_id contains a unique identifier for each artist and not the name of the artist.
This is okay because it is the same thing as encoding the artist name as a number.

In [22]:
artist_df = pd.read_excel('data/songs_warehouse.xlsx', sheet_name='artist_of_song')
# rename the column song_id to id_song
artist_df = artist_df.rename(columns={'song_id': 'id_song'})
artist_df

,id_song,artist_id
0,1,481
1,1,994
2,1,1339
3,1,1516
4,1,2667
...,...,...
8839,5497,1869
8840,5497,1881
8841,5498,25
8842,5498,1459


## Get the average popularity of each song

In [23]:
# find the average popularity of each song
average_popularity = df2.groupby('song')['popularity'].mean()
# rename the column song to id_song and the column popularity to average_popularity
average_popularity = average_popularity.reset_index().rename(columns={'song': 'id_song', 'popularity': 'average_popularity'})
average_popularity

,id_song,average_popularity
0,1,62.000000
1,2,33.000000
2,3,87.615385
3,4,76.179487
4,5,65.857143
...,...,...
5493,5495,76.535714
5494,5496,29.000000
5495,5497,37.500000
5496,5498,62.166667


## Remove outliers that may skew the model

In [24]:
# remove the rows where the average_popularity is 0
average_popularity = average_popularity[average_popularity['average_popularity'] > 10]

# remove the rows where the average_popularity is lower than 90
average_popularity = average_popularity[average_popularity['average_popularity'] < 90]

## Merge the dataframes and drop unnecessary columns

In [25]:
# merge average_popularity with df on song 
merged_df = df.merge(average_popularity, on='id_song')

# merge artist_df with merged_df on song
merged_df = merged_df.merge(artist_df, on='id_song')

# break the column label_album_release_date into year, month, day
merged_df['year'] = merged_df['label_album_release_date'].dt.year
merged_df['month'] = merged_df['label_album_release_date'].dt.month
merged_df['day'] = merged_df['label_album_release_date'].dt.day
# drop the column label_album_release_date
merged_df = merged_df.drop(columns=['label_album_release_date' ])

In [26]:
# drop the columns label_album_name,  label_mode
merged_df = merged_df.drop(columns=['label_album_name', 'label_mode'])
merged_df

,id_song,label_spotify_id,label_name,label_is_explicit,label_duration_ms,label_danceability,label_energy,label_key,label_loudness,label_speechiness,...,label_instrumentalness,label_liveness,label_valence,label_tempo,label_time_signature,average_popularity,artist_id,year,month,day
0,42,03Dpt8Z4Zww4NGJb8503zb,Do They Know It's Christmas? - 2014,False,228916,0.626,0.541,0,-7.615,0.0308,...,0.000000,0.1190,0.255,112.000999,4,55.000000,1913,2018.0,11.0,9.0
1,128,0a0zPUrwviAua4IhhaYUsP,Ajándék,False,268240,0.668,0.864,0,-4.404,0.0343,...,0.000000,0.1050,0.593,102.014000,4,48.000000,3294,2009.0,1.0,1.0
2,128,0a0zPUrwviAua4IhhaYUsP,Ajándék,False,268240,0.668,0.864,0,-4.404,0.0343,...,0.000000,0.1050,0.593,102.014000,4,48.000000,4089,2009.0,1.0,1.0
3,182,0cVyQfDyRnMJ0V3rjjdlU3,Lil Boo Thang,False,114233,0.850,0.699,0,-3.292,0.0776,...,0.000000,0.3200,0.915,114.481003,4,86.000000,1751,2023.0,8.0,18.0
4,270,0gq4UgDPGFdqpsWshU7dmv,Vanavond (Uit M'n Bol),False,166560,0.799,0.705,0,-7.582,0.0698,...,0.000000,0.4260,0.880,106.978996,4,62.000000,509,2022.0,3.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8468,5433,7uHFrsclJjAhONLcc8grZU,Шукав тебе. Знайшов тебе,False,167999,0.711,0.600,0,-10.639,0.1410,...,0.000001,0.0843,0.201,112.125999,4,58.136986,3005,2021.0,7.0,30.0
8469,5433,7uHFrsclJjAhONLcc8grZU,Шукав тебе. Знайшов тебе,False,167999,0.711,0.600,0,-10.639,0.1410,...,0.000001,0.0843,0.201,112.125999,4,58.136986,3404,2021.0,7.0,30.0
8470,5459,7wcjeveluiW5djtxLCxNSx,Thath'Indawo (Live),False,475664,0.380,0.646,0,-8.379,0.0678,...,0.000001,0.1350,0.315,105.845001,4,56.000000,4066,2022.0,9.0,16.0
8471,5459,7wcjeveluiW5djtxLCxNSx,Thath'Indawo (Live),False,475664,0.380,0.646,0,-8.379,0.0678,...,0.000001,0.1350,0.315,105.845001,4,56.000000,4242,2022.0,9.0,16.0


## Set the dependent and independent variables

In [27]:
# use xgboost to predict the average popularity of a song
X = merged_df.drop(columns=['average_popularity', 'id_song', 'label_name', 'label_spotify_id'])
y = merged_df['average_popularity']

## Split the data into training and testing sets
We will use 10% of the data for testing and 90% for training.

In [28]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1000)

## Create and train the model

In [29]:
# initialize the XGBoost regressor
xgb_reg = xgb.XGBRegressor()
# train the model
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Get the feature importance

In [30]:
# get the feature names
feature_names = X_train.columns
# create a dataframe with feature names and feature importance
feature_importance_df = pd.DataFrame({'feature_names': feature_names, 'feature_importance': xgb_reg.feature_importances_})
# sort the dataframe by feature_importance
feature_importance_df = feature_importance_df.sort_values('feature_importance', ascending=False)
feature_importance_df

,feature_names,feature_importance
15,month,0.241368
14,year,0.142258
16,day,0.086468
5,label_loudness,0.057707
7,label_acousticness,0.047068
8,label_instrumentalness,0.044525
11,label_tempo,0.041449
6,label_speechiness,0.040630
9,label_liveness,0.039392
10,label_valence,0.038931


## Make predictions on the test set

In [31]:
# make predictions on X_test
predictions = xgb_reg.predict(X_test)

## Evaluate the model

In [32]:
# rmse (root mean squared error)
rmse = mean_squared_error(y_test, predictions, squared=False)
print('The rmse is:', round(rmse, 2))
# mae (mean absolute error)
mae = (abs(predictions - y_test)).mean()
print('The mae is:', round(mae, 2))
# mape (mean absolute percentage error)
mape = 100 * (abs(predictions - y_test) / y_test).mean()
print('The mape is:', round(mape, 2))

The rmse is: 10.28
The mae is: 7.57
The mape is: 14.98


## Compare the predictions with the actual values

In [33]:
# put into a dataframe
predictions_df = pd.DataFrame(predictions)
predictions_df
# merge predictions_df with y_test
predictions_df = predictions_df.merge(y_test, left_index=True, right_index=True)

In [34]:
# rename the columns to predicted and actual
predictions_df = predictions_df.rename(columns={0: 'predicted', 'average_popularity': 'actual'})
predictions_df

,predicted,actual
1,68.843620,48.000000
6,56.923599,62.000000
18,27.905424,78.921875
31,54.233852,72.000000
42,66.686066,45.361702
...,...,...
803,61.944675,79.378378
804,65.557625,79.378378
820,73.608948,12.000000
826,30.177361,43.000000


## Show the rows where the 2 columns have a difference of less than 20

In [35]:
# show the rows where the 2 columns have a difference of less than 20
predictions_df[abs(predictions_df['predicted'] - predictions_df['actual']) < 20]

,predicted,actual
6,56.923599,62.000000
31,54.233852,72.000000
57,66.952652,49.272727
65,62.650089,52.000000
76,63.107349,75.000000
78,55.235771,56.000000
147,85.282066,76.775862
152,65.441010,52.000000
154,76.769211,58.525424
163,67.037758,59.956522


In [36]:
# print the percentage of rows where the 2 columns have a difference of less than 20
percentage = len(predictions_df[abs(predictions_df['predicted'] - predictions_df['actual']) < 20]) / len(predictions_df)
print('The percentage with a difference of less than 20 is:', round(percentage * 100, 2), '%')

The percentage with a difference of less than 20 is: 70.24 %
